In [7]:
import pandas as pd

In [8]:
fn = "../tokens/scrape_erc20/erc20.jsonlines"

with open(fn) as f:
    content = f.read()

print(content.split("\n")[0])

{"index": "6", "name": "Matic Token (MATIC)", "address": "0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0", "price_in_usd": "0.7015", "volume": "310862957.00", "market_cap_circulating": "6537801645.00", "market_cap_onchain": "7015208265.94", "holders": "618802", "market_cap_fully_diluted": "7015208265.94", "total_supply": "10000000000", "decimals": "18", "official_website": "polygon.technology"}


In [9]:
erc20_df = pd.read_json(fn, lines=True)

In [10]:
erc20_df.sample()

,index,name,address,price_in_usd,volume,market_cap_circulating,market_cap_onchain,holders,market_cap_fully_diluted,total_supply,decimals,official_website
72,1208,Crystal Clear Token (CCT),0x336f646f87d9f6bc6ed42dd46e8b3fd9dbd15c22,0.001,1,5831,6815.42,1213,6815.42,6924060,18,crystal-clear.io


In [11]:
erc20_df[erc20_df["market_cap_circulating"] > 1_000_000].count().values

array([758, 758, 758, 758, 758, 758, 758, 758, 758, 758, 758, 758],
      dtype=int64)

In [12]:
erc20_df[erc20_df["market_cap_circulating"] > 10_000_000].count().values

array([436, 436, 436, 436, 436, 436, 436, 436, 436, 436, 436, 436],
      dtype=int64)

In [13]:
erc20_df[erc20_df["market_cap_circulating"] > 100_000_000].count().values

array([132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132],
      dtype=int64)

In [14]:
erc20_df[erc20_df["market_cap_circulating"] > 1_000_000_000].count().values

array([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27], dtype=int64)

In [15]:
pom_fn = "X:\\pomabuster\\pom\\pom2.jsonl"
arb_fn = "X:\\pomabuster\\arb\\arb2.jsonl"

pom_df = pd.read_json(pom_fn, lines=True)
arb_df = pd.read_json(arb_fn, lines=True)

In [16]:
from dataclasses import dataclass

@dataclass
class Log:
    sender: str
    receiver: str
    amount: str
    asset: str
    idx: int
    
@dataclass
class Trade:
    operator: str
    recipient: str
    pool: str
    asset_in: str
    asset_out: str
    amount_in: str
    amount_out: str
    log_idx: list

def parse_record(record):
    assets = record.token_address
    senders = record.from_address
    receivers = record.to_address
    amounts = record.value
    logs = []
    for i in range(len(assets)):      
        log_dict = {
            "idx": i,
            "asset": assets[i],
            "sender": senders[i],
            "receiver": receivers[i],
            "amount": amounts[i],
        }
        logs.append(Log(**log_dict))
    return logs

In [17]:
high_value_erc20 = erc20_df[erc20_df["market_cap_circulating"] > 1_000_000_000]["address"].values

In [18]:
pom_df.sample()

,block_number,transaction_hash,token_address,from_address,to_address,value,operator,recipient,is_trade
52343,12685270,0x4bb4d662f4858cd5262411548a7a2b97ca2dad39a46c...,"[0x6b175474e89094c44da98b954eedeac495271d0f, 0...","[0xaae0633e15200bc9c50d45cd762477d268e126bd, 0...","[0x1c073d5045b1abb6924d5f0f8b2f667b1653a4c3, 0...","[340736000000000000000000, 3407360000000000000...",0xaae0633e15200bc9c50d45cd762477d268e126bd,0x9a67f1940164d0318612b497e8e6038f902a00a4,True


In [19]:
len(pom_df)

58957

In [20]:
# filter out POM first
#new_pom_df = pom_df[pom_df["pom_df"]]
from tqdm import tqdm

included = []
for _, pom in tqdm(pom_df.iterrows()):
    flag = False
    pom_logs = parse_record(pom)
    pom_assets = [log.asset for log in pom_logs]
    for asset in pom_assets:
        if asset in high_value_erc20:
            flag = True
    included.append(flag)


58957it [00:07, 7948.83it/s]


In [21]:
sum(included)

57008

In [23]:
pom_df["include"] = included

In [24]:
new_pom_df = pom_df[pom_df["include"] == True]

In [25]:
from tqdm import tqdm

link = []
for _, pom in tqdm(new_pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

57008it [02:10, 437.61it/s]


In [26]:
print(len(link))

16323


In [ ]:
high_value_erc20 = erc20_df[erc20_df["market_cap_circulating"] > 1_00_000_000]["address"].values

included = []
for _, pom in tqdm(pom_df.iterrows()):
    flag = False
    pom_logs = parse_record(pom)
    pom_assets = [log.asset for log in pom_logs]
    for asset in pom_assets:
        if asset in high_value_erc20:
            flag = True
    included.append(flag)

new_pom_df = pom_df[pom_df["include"] == True]

link = []
for _, pom in tqdm(new_pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

print(len(link))

In [ ]:
high_value_erc20 = erc20_df[erc20_df["market_cap_circulating"] > 10_000_000]["address"].values

included = []
for _, pom in tqdm(pom_df.iterrows()):
    flag = False
    pom_logs = parse_record(pom)
    pom_assets = [log.asset for log in pom_logs]
    for asset in pom_assets:
        if asset in high_value_erc20:
            flag = True
    included.append(flag)

new_pom_df = pom_df[pom_df["include"] == True]

print(sum(included))

In [ ]:
link = []
for _, pom in tqdm(pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

print(len(link))

In [27]:
link_df = pd.DataFrame(link, columns=["pom_transaction_hash","arb_transaction_hash"])
pom_df.rename(columns={"transaction_hash": "pom_transaction_hash"}, inplace=True)
arb_df.rename(columns={"transaction_hash": "arb_transaction_hash"}, inplace=True)
merged_pom_df = pd.merge(pom_df, link_df, on="pom_transaction_hash")

In [28]:
merged_pom_df.head(5)

,block_number,pom_transaction_hash,token_address,from_address,to_address,value,operator,recipient,is_trade,include,arb_transaction_hash
0,12465434,0x5dccf7377422eaa76613b770b088d7edf7c2bcfae00e...,"[0x03ab458634910aad20ef5f1c8ee96f1d6ac54919, 0...","[0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d, 0...","[0x22f9dcf4647084d6c31b2765f6910cd85c178c18, 0...","[100000000000000000000000, 5645576516772765803...",0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d,0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d,True,True,0x146d1444865b017ea9635ff1c0c7a7c15516fa409e30...
1,12923530,0xca0c565f84ed6a3f984972aedaf74ebf98d173e950f2...,"[0x03ab458634910aad20ef5f1c8ee96f1d6ac54919, 0...","[0x92e74e50edce4573d181c03ca889f72640cd3a94, 0...","[0x22f9dcf4647084d6c31b2765f6910cd85c178c18, 0...","[18000000000000000000000, 54013118772, 1800000...",0x92e74e50edce4573d181c03ca889f72640cd3a94,0x92e74e50edce4573d181c03ca889f72640cd3a94,True,True,0x595c9b15fa0d2e20d6e9a2984db7689d18b40e316773...
2,12367976,0xaae9835fbdf73c594693d8c99ee6fd3a69de6c1a5244...,"[0x03ab458634910aad20ef5f1c8ee96f1d6ac54919, 0...","[0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d, 0...","[0x8ae720a71622e824f576b4a8c03031066548a3b1, 0...","[35504727364592962230998, 31420791200966009936...",0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d,0xd4ad5d62dce1a8bf661777a5c1df79bd12ac8f1d,True,True,0x18945e46cd925cbe8b1c0f64ba852ffe06b7a9bbcbea...
3,12194336,0x99fa6f5af9c414d72554f2579f4e5239ede256c45a99...,"[0x03ab458634910aad20ef5f1c8ee96f1d6ac54919, 0...","[0x7f977c4cff92c055173b865bcbd8158161ab95a9, 0...","[0x8ae720a71622e824f576b4a8c03031066548a3b1, 0...","[70000000000000000000000, 10534880207223427197...",0x7f977c4cff92c055173b865bcbd8158161ab95a9,0x7f977c4cff92c055173b865bcbd8158161ab95a9,True,True,0xfbe9dc34fd3d63a57b574c920fd223b71f2b06c3c9c1...
4,12531411,0xa22d3350fbde7901b3d355260ebb0a12b83ffb406bf6...,"[0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c, 0...","[0x649e74b1cebafe73250b5bd1416dc3c9040680ac, 0...","[0xfd3dfb524b2da40c8a6d703c62be36b5d8540626, 0...","[480000000045371620249192, 4800000000453716202...",0x649e74b1cebafe73250b5bd1416dc3c9040680ac,0x649e74b1cebafe73250b5bd1416dc3c9040680ac,True,True,0x3645fdd74f85b482ee0e35cb677371edf8e4db3aa224...


In [ ]:
# start 67.9hr, 434
# 2021-06-01 00:00 12545219, 89426448, 13.9hr
# 2022-01-01 00:00 13916166, 108462953, 16.8 hr
# 2022-06-01 00:00 14881677, 57243186, 8.9hr
# 2023-01-01 00:00 16308190, 92805628, 14.4 hr
# end,                       88073705, 13.9 hr

In [ ]:
len(merged_pom_df[merged_pom_df["block_number"] < 12545219])

In [ ]:
len(merged_pom_df[(12545219 <= merged_pom_df["block_number"]) & (merged_pom_df["block_number"] < 13916166)])

In [ ]:
len(merged_pom_df[(13916166 <= merged_pom_df["block_number"]) & (merged_pom_df["block_number"] < 14881677)])

In [ ]:
len(merged_pom_df[(14881677 <= merged_pom_df["block_number"]) & (merged_pom_df["block_number"] < 16308190)])

In [ ]:
len(merged_pom_df[(16308190 <= merged_pom_df["block_number"])])

In [ ]:
# start 67.9hr, 434
# 2021-03-01 00:00 11948960
# 2021-06-01 00:00 12545219
# 2021-09-01 00:00 13136427
# 2022-01-01 00:00 13916166
# 2022-03-01 00:00 14297759
# 2022-06-01 00:00 14881677
# 2022-09-01 00:00 15449618
# 2023-01-01 00:00 16308190
# 2023-03-01 00:00 16730072
# end,                       88073705, 13.9 hr

In [29]:
block_range = [0, 11948960, 12545219, 13136427, 13916166, 14297759, 14881677, 15449618, 16308190, 16_730_072, 99_999_999]
block_range_lshift = block_range[1:] + [99_999_999]
block_range_zip = list(zip(block_range, block_range_lshift))[:-1]
block_range_zip

[(0, 11948960),
 (11948960, 12545219),
 (12545219, 13136427),
 (13136427, 13916166),
 (13916166, 14297759),
 (14297759, 14881677),
 (14881677, 15449618),
 (15449618, 16308190),
 (16308190, 16730072),
 (16730072, 99999999)]

In [30]:
for (interval_i, interval_j) in block_range_zip:
    poma_number = len(merged_pom_df[(interval_i <= merged_pom_df["block_number"]) & (merged_pom_df["block_number"] < interval_j)])
    print(f"[{interval_i},{interval_j}]: {poma_number}")

[0,11948960]: 1409
[11948960,12545219]: 3258
[12545219,13136427]: 2495
[13136427,13916166]: 1726
[13916166,14297759]: 458
[14297759,14881677]: 1306
[14881677,15449618]: 2426
[15449618,16308190]: 2256
[16308190,16730072]: 401
[16730072,99999999]: 588


In [ ]:
# 2021-03-01 11948960 1409
# 2021-06-01 12545219 3258
# 2021-09-01 13136427 2495
# 2022-01-01 13916166 1726
# 2022-03-01 14297759 458
# 2022-06-01 14881677 1306
# 2022-09-01 15449618 2426
# 2023-01-01 16308190 2256
# 2023-03-01 16730072 401
# 2023-05-31 99999999 588

In [35]:
len(link_5)

24313

In [34]:
link = []
for _, pom in tqdm(pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 10)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link.append((pom["pom_transaction_hash"], arb.iloc[0]["arb_transaction_hash"]))

58957it [02:24, 406.68it/s]


In [36]:
link_5 = []
for _, pom in tqdm(pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 5)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link_5.append((pom["pom_transaction_hash"], arb.iloc[0]["arb_transaction_hash"]))

58957it [02:22, 415.16it/s]


In [37]:
len(link_5)

20503

In [38]:
import pandas as pd

fn = "../tokens/scrape_erc20/erc20.jsonlines"
token_df = pd.read_json(fn, lines=True)

token_supply = {}
for _, row in token_df.iterrows():
    token_supply[row["address"]] = int(row["total_supply"])
    
token_decimal = {}
for _, row in token_df.iterrows():
    token_decimal[row["address"]] = int(row["decimals"])

In [39]:
filepath = "drf_000"
poma = pd.read_csv(filepath)
poma.sample()


,0
312,0x6bfcb835d870baab03a5480d4844873cc26229ba8471...


In [40]:
import itertools

ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
ZERO_AMOUNT = "0"
def is_transfer_normal(log):
    return (log.sender != log.receiver 
        and log.sender != ZERO_ADDRESS
        and log.receiver != ZERO_ADDRESS
        and log.amount != ZERO_AMOUNT)

def is_transfer_minting(log):
    return (log.sender == ZERO_ADDRESS and log.amount != ZERO_AMOUNT)

def is_transfer_burning(log):
    return (log.receiver == ZERO_ADDRESS and log.amount != ZERO_AMOUNT)

def has_transfer_normal(logs):
    return [log for log in logs if is_transfer_normal(log)]

def has_transfer_minting(logs):
    return [log for log in logs if is_transfer_minting(log)]

def has_transfer_burning(logs):
    return [log for log in logs if is_transfer_burning(log)]

def has_liquidity_mining(logs):
    ts = has_transfer_normal(logs)
    tms = has_transfer_minting(logs)
    if not ts or not tms:
        return None
    for t, tm in pairwise(ts, tms):
        if t and tm and t.asset != tm.asset:
            trade_dict = {"operator": t.sender, 
                    "recipient": tm.receiver, 
                    "pool": t.receiver, 
                    "asset_in": t.asset,
                    "asset_out": tm.asset,
                    "amount_in": t.amount,
                    "amount_out": tm.amount,
                    "log_idx": [t.idx, tm.idx]}
            yield Trade(**trade_dict)
        return None

def has_liquidity_cancel(logs):
    ts = has_transfer_normal(logs)
    tbs = has_transfer_burning(logs)
    if not ts or not tbs:
        return None
    for t, tb in pairwise(ts, tbs):
        if t and tb and t.asset != tb.asset:
            trade_dict =  {"operator": tb.sender, 
                    "recipient": t.receiver, 
                    "pool": t.sender, 
                    "asset_in": tb.asset,
                    "asset_out": t.asset,
                    "amount_in": tb.amount,
                    "amount_out": t.amount,
                    "log_idx": [t.idx, tb.idx]}
            yield Trade(**trade_dict)
    return None

def pairwise(iterable_1, iterable_2=None):
    if iterable_2 is None:
        return itertools.product(iterable_1, iterable_1)
    return itertools.product(iterable_1, iterable_2)

def has_trade(logs):
    t = has_transfer_normal(logs)
    if not t:
        return None
    for t1, t2 in pairwise(t):
        if t1 and t2 and t1.asset != t2.asset and t1.receiver == t2.sender:
            trade_dict = {"operator": t1.sender, 
                    "recipient": t2.receiver, 
                    "pool": t1.receiver, 
                    "asset_in": t1.asset,
                    "asset_out": t2.asset,
                    "amount_in": t1.amount,
                    "amount_out": t2.amount,
                    "log_idx": [t1.idx, t2.idx]}
            yield Trade(**trade_dict)
    return None

def has_trade_alt(logs):
    lms = has_liquidity_mining(logs)
    lcs = has_liquidity_cancel(logs)
    if not lms or lcs:
        return None
    for lm, lc in pairwise(lms, lcs):
        if (lm and lc 
            and lm.recipient == lc.operator 
            and lm.asset_in != lc.asset_out 
            and lm.asset_out != lc.asset_in):
            trade_dict =  {"operator": lm.operator, 
                    "recipient": lc.recipient, 
                    "pool": lm.recipient, 
                    "asset_in": lm.asset_in,
                    "asset_out": lc.asset_out,
                    "amount_in": lm.amount_in,
                    "amount_out": lc.amount_out,
                    "log_idx": [lm.idx, lc.idx]}
            yield Trade(**trade_dict)
    return None



In [41]:
def parse_record(record):
    assets = record.token_address
    senders = record.from_address
    receivers = record.to_address
    amounts = record.value
    logs = []
    for i in range(len(assets)):      
        log_dict = {
            "idx": i,
            "asset": assets[i],
            "sender": senders[i],
            "receiver": receivers[i],
            "amount": amounts[i],
        }
        logs.append(Log(**log_dict))
    return logs

In [42]:
PERCENTAGE = 0.02

def has_trade_huge(trades, percentage=PERCENTAGE):
    for trade in trades:
        if (int(trade.amount_in) / (10 ** token_decimal[trade.asset_in]) >= token_supply[trade.asset_in] * PERCENTAGE 
                or int(trade.amount_out) / (10 ** token_decimal[trade.asset_out]) >= token_supply[trade.asset_out] * PERCENTAGE ):
            yield trade
            
def has_wash_trade(trades, percentage=PERCENTAGE):
    total_amount = {}
    for trade in trades:
        if trade.asset_out not in total_amount:
            total_amount[trade.asset_out] = int(trade.amount_out)
        else:
            total_amount[trade.asset_out] += int(trade.amount_out)
    for token, amount in total_amount.items():
        if total_amount[token] / (10 ** token_decimal[token]) >= token_supply[token] * PERCENTAGE:
            return True
    return False

In [43]:
def is_direct_pm(logs, percentage):
    trades = list(has_trade(logs)) + list(has_trade_alt(logs))
    huge_trades = list(has_trade_huge(trades, percentage))
    if huge_trades:
        return True
    if has_wash_trade(trades, percentage):
        return True
    return False

In [46]:
from tqdm import tqdm

PERCENTAGE = 0.015
output_path  ="X:\pomabuster\pom\pom_015.jsonl"

fn_template = "X:\pomabuster\swap-new\swap-000000000" # 000 - 399 # swap-000000000000
for i in tqdm(range(400)):
    if len(str(i)) == 1:
        file_number = "00" + str(i)
    elif len(str(i)) == 2:
        file_number = "0" + str(i)
    else:
        file_number = str(i)
    # print(file_number)
    df = pd.read_json(fn_template + file_number, lines=True)
    pm_txns = []
    for _, record in df.iterrows():
        logs = parse_record(record)
        if is_direct_pm(logs, PERCENTAGE):
            pm_txns.append(record.transaction_hash)
    pm_df = df.loc[df["transaction_hash"].isin(pm_txns)]
    with open(output_path, "a") as f:
        f.write(pm_df.to_json(orient='records', lines=True))

100%|███████████████████████████████████████████████████████████████████████| 400/400 [1:02:46<00:00,  9.42s/it]


In [47]:
pom_fn = "X:\\pomabuster\\pom\\pom_015.jsonl"
arb_fn = "X:\\pomabuster\\arb\\arb2.jsonl"

pom_015_df = pd.read_json(pom_fn, lines=True)
arb_df = pd.read_json(arb_fn, lines=True)

In [57]:
link_015 = []
for _, pom in tqdm(pom_015_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link_015.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

78975it [03:39, 359.03it/s]


In [58]:
len(link_015)

23146

In [50]:
PERCENTAGE = 0.025
output_path  ="X:\pomabuster\pom\pom_025.jsonl"

fn_template = "X:\pomabuster\swap-new\swap-000000000" # 000 - 399 # swap-000000000000
for i in tqdm(range(400)):
    if len(str(i)) == 1:
        file_number = "00" + str(i)
    elif len(str(i)) == 2:
        file_number = "0" + str(i)
    else:
        file_number = str(i)
    # print(file_number)
    df = pd.read_json(fn_template + file_number, lines=True)
    pm_txns = []
    for _, record in df.iterrows():
        logs = parse_record(record)
        if is_direct_pm(logs, PERCENTAGE):
            pm_txns.append(record.transaction_hash)
    pm_df = df.loc[df["transaction_hash"].isin(pm_txns)]
    with open(output_path, "a") as f:
        f.write(pm_df.to_json(orient='records', lines=True))

100%|███████████████████████████████████████████████████████████████████████| 400/400 [1:02:54<00:00,  9.44s/it]


In [59]:
pom_fn = "X:\\pomabuster\\pom\\pom_025.jsonl"
arb_fn = "X:\\pomabuster\\arb\\arb2.jsonl"

pom_025_df = pd.read_json(pom_fn, lines=True)

link_025 = []
for _, pom in tqdm(pom_025_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link_025.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

46082it [01:46, 434.67it/s]


In [60]:
len(link_025)

12423